In [1]:
import numpy as np
from scipy.optimize import brentq
import geomdl
# Importing NURBS module
from geomdl import NURBS
# Importing visualization module
from geomdl.visualization import VisMPL as vis
# If there is an error saying: "AttributeError: module 'numpy' has no attribute 'float'.", we need to degrade numpy to 1.23.5
# conda install numpy==1.23.5
import gmsh

## Pamameter settings

In [2]:
r = np.sqrt(5) / 10 # radius of porous
N = 5 # the number of subdivision of half porous
ms = 2.0 * np.pi * r / (2 * N) # mesh size

## Create outer boundaries

In [3]:
porous = "ASCIIV4/Porous/porous.msh"
gmsh.initialize()
gmsh.model.add(porous)

crvlp = []
outer_bd = []
grid_idx = 1
line_idx = 1
crvlp_idx = 1

# 1st: left partial bottom boundary
gmsh.model.geo.addPoint(-1.0, -1.0, 0, ms, grid_idx)
grid_idx = grid_idx + 1
gmsh.model.geo.addPoint(0.5 - r, -1.0, 0, ms, grid_idx)
gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)

outer_bd.append(line_idx)
grid_idx = grid_idx + 1
line_idx = line_idx + 1

# 2nd: half porous at bottom
p1 = []
l1 = []
for k in range(N):
    x = 0.5 - r * np.cos((k + 1) * np.pi / N)
    y = -1.0 + r * np.sin((k + 1) * np.pi / N)
    gmsh.model.geo.addPoint(x, y, 0, ms, grid_idx)
    gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
    l1.append(line_idx)
    p1.append([grid_idx - 1, grid_idx, 0.5 + 0.5 * k / N, 0.5 + 0.5 * (k + 1) / N])
    outer_bd.append(line_idx)
    grid_idx = grid_idx + 1
    line_idx = line_idx + 1

# 3rd: right partial bottom boundary
gmsh.model.geo.addPoint(1.0, -1.0, 0, ms, grid_idx)
gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
br = line_idx
outer_bd.append(line_idx)
grid_idx = grid_idx + 1
line_idx = line_idx + 1

# 4th: right boundary
gmsh.model.geo.addPoint(1.0, 1.0, 0, ms, grid_idx)
gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
outer_bd.append(line_idx)
grid_idx = grid_idx + 1
line_idx = line_idx + 1

# 5th: right partial top boundary
gmsh.model.geo.addPoint(0.5 + r, 1.0, 0, ms, grid_idx)
gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
outer_bd.append(line_idx)
grid_idx = grid_idx + 1
line_idx = line_idx + 1

# 7th: half porous at top
p2 = []
l2 = []
for k in range(N):
    x = 0.5 + r * np.cos((k + 1) * np.pi / N)
    y = 1.0 - r * np.sin((k + 1) * np.pi / N)
    gmsh.model.geo.addPoint(x, y, 0, ms, grid_idx)
    gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
    l2.append(line_idx)
    p2.append([grid_idx - 1, grid_idx, 0.5 * k / N, 0.5 * (k + 1) / N])
    outer_bd.append(line_idx)
    grid_idx = grid_idx + 1
    line_idx = line_idx + 1

# 8th: left partial top boundary
gmsh.model.geo.addPoint(-1.0, 1.0, 0, ms, grid_idx)
gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
tl = line_idx
outer_bd.append(line_idx)
line_idx = line_idx + 1

# 9th: left boundary
gmsh.model.geo.addLine(grid_idx, 1, line_idx)
outer_bd.append(line_idx)
grid_idx = grid_idx + 1
line_idx = line_idx + 1

gmsh.model.geo.addCurveLoop(outer_bd, crvlp_idx)
crvlp.append(crvlp_idx)
crvlp_idx = crvlp_idx + 1

# 1st interior porous: top left
p3 = []
l3 = []
s1 = grid_idx
gmsh.model.geo.addPoint(- 0.5 + r, 0.5, 0, ms, grid_idx)
grid_idx = grid_idx + 1
for k in range(1, 2 * N):
    x = - 0.5 + r * np.cos(k * np.pi / N)
    y = 0.5 - r * np.sin(k * np.pi / N)
    gmsh.model.geo.addPoint(x, y, 0, ms, grid_idx)
    gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
    l3.append(line_idx)
    p3.append([grid_idx - 1, grid_idx, 0.5 * k / N, 0.5 * (k + 1) / N])
    outer_bd.append(line_idx)
    grid_idx = grid_idx + 1
    line_idx = line_idx + 1
gmsh.model.geo.addLine(grid_idx - 1, s1, line_idx)
l3.append(line_idx)
p3.append([grid_idx - 1, grid_idx, 0.5 * (2 * N-1) / N, 1])
outer_bd.append(line_idx)
line_idx = line_idx + 1
gmsh.model.geo.addCurveLoop(l3, crvlp_idx)
crvlp.append(crvlp_idx)
crvlp_idx = crvlp_idx + 1

# 2nd interior porous: top left
p4 = []
l4 = []
s2 = grid_idx
gmsh.model.geo.addPoint(- 0.5 + r, - 0.5, 0, ms, grid_idx)
grid_idx = grid_idx + 1
for k in range(1, 2 * N):
    x = - 0.5 + r * np.cos(k * np.pi / N)
    y = - 0.5 - r * np.sin(k * np.pi / N)
    gmsh.model.geo.addPoint(x, y, 0, ms, grid_idx)
    gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
    l4.append(line_idx)
    p4.append([grid_idx - 1, grid_idx, 0.5 * k / N, 0.5 * (k + 1) / N])
    outer_bd.append(line_idx)
    grid_idx = grid_idx + 1
    line_idx = line_idx + 1
gmsh.model.geo.addLine(grid_idx - 1, s2, line_idx)
l4.append(line_idx)
p4.append([grid_idx - 1, grid_idx, 0.5 * (2 * N-1) / N, 1])
outer_bd.append(line_idx)
line_idx = line_idx + 1
gmsh.model.geo.addCurveLoop(l4, crvlp_idx)
crvlp.append(crvlp_idx)
crvlp_idx = crvlp_idx + 1

# 3rd interior porous: top left
p5 = []
l5 = []
s3 = grid_idx
gmsh.model.geo.addPoint(0.5 + r, 0, 0, ms, grid_idx)
grid_idx = grid_idx + 1
for k in range(1, 2 * N):
    x = 0.5 + r * np.cos(k * np.pi / N)
    y = - r * np.sin(k * np.pi / N)
    gmsh.model.geo.addPoint(x, y, 0, ms, grid_idx)
    gmsh.model.geo.addLine(grid_idx - 1, grid_idx, line_idx)
    l5.append(line_idx)
    p5.append([grid_idx - 1, grid_idx, 0.5 * k / N, 0.5 * (k + 1) / N])
    outer_bd.append(line_idx)
    grid_idx = grid_idx + 1
    line_idx = line_idx + 1
gmsh.model.geo.addLine(grid_idx - 1, s3, line_idx)
l5.append(line_idx)
p5.append([grid_idx - 1, grid_idx, 0.5 * (2 * N-1) / N, 1])
outer_bd.append(line_idx)
line_idx = line_idx + 1
gmsh.model.geo.addCurveLoop(l5, crvlp_idx)
crvlp.append(crvlp_idx)

gmsh.model.addPhysicalGroup(1, [1], name = "bottom left")
gmsh.model.addPhysicalGroup(1, l1, name = "bottom concave")
gmsh.model.addPhysicalGroup(1, [br], name = "bottom right")
gmsh.model.addPhysicalGroup(1, [br + 1], name = "right")
gmsh.model.addPhysicalGroup(1, [br + 2], name = "top right")
gmsh.model.addPhysicalGroup(1, l2, name = "top concave")
gmsh.model.addPhysicalGroup(1, [tl], name = "top left")
gmsh.model.addPhysicalGroup(1, [tl + 1], name = "left")
gmsh.model.addPhysicalGroup(1, l3, name = "interior porous 1")
gmsh.model.addPhysicalGroup(1, l4, name = "interior porous 2")
gmsh.model.addPhysicalGroup(1, l5, name = "interior porous 3")

gmsh.model.geo.addPlaneSurface(crvlp, 1)
# gmsh.model.geo.addPlaneSurface([crvlp[1]], 2)
# gmsh.model.geo.addPlaneSurface([crvlp[2]], 3)
# gmsh.model.geo.addPlaneSurface([crvlp[3]], 4)
gmsh.model.addPhysicalGroup(2, [1], name="2D Domain")

gmsh.model.geo.synchronize()

# translation = [1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
# gmsh.model.mesh.setPeriodic(1, [8], [2], translation)

gmsh.model.mesh.generate(2)

gmsh.write(porous)

gmsh.fltk.run()
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 20%] Meshing curve 7 (Line)
Info    : [ 20%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Line)
Info    : [ 20%] Meshing curve 10 (Line)
Info    : [ 30%] Meshing curve 11 (Line)
Info    : [ 30%] Meshing curve 12 (Line)
Info    : [ 30%] Meshing curve 13 (Line)
Info    : [ 30%] Meshing curve 14 (Line)
Info    : [ 40%] Meshing curve 15 (Line)
Info    : [ 40%] Meshing curve 16 (Line)
Info    : [ 40%] Meshing curve 17 (Line)
Info    : [ 40%] Meshing curve 18 (Line)
Info    : [ 40%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 50%] Meshing curve 21 (Line)
Info    : [ 50%] Meshing curve 22 (Line)
Info    : [ 50%] Meshing curve 23 (Line)
Info    : [ 60%] Meshing curve 24 (Line)
I